In [136]:
import pandas as pd
import numpy as np
import datetime as dt
from numpy.random import choice

In [206]:
#df=pd.read_csv('DATA/dashmote_id_&_platframts_ids_2.csv')
df=pd.read_csv('DATA/dashmote_id_&_platframts_ids_3.csv')

len(df)

25320

In [207]:
df.head()

,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id
0,89977249-e023-405a-b533-3c1f33f804a2,5f04302c-68b1-3a83-8df9-0a05262b7200,NaN,40408a01-d9b3-443b-8e9e-53f8afb513f3,c26dafda-0437-4a47-9f79-6fb45541f1ea
1,9e7b397d-d615-4fb1-95d7-9d04d07765fd,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,NaN,NaN,1eb55afc-2faf-4c81-b6ff-5cdd4aa0f107
2,1e7ca034-a7c2-4a7c-8b3e-65bb4bf54da5,a80efa26-9e1d-31e3-88a5-b68d983be5c5,NaN,NaN,6f6f8fc6-1a3d-4e21-bfdc-45c1aa8b7577
3,9e829615-d0e6-446e-89cc-8940cb76f452,ca013261-a68e-3cbe-86fc-57c59f09940b,NaN,dbe938c5-eea7-4b0a-811d-145c679ba64a,ae702a3c-c50e-4c80-8db1-15e12a6837ad
4,1fc69820-eca5-4a86-b50b-db666ed2aef8,2326296b-c3fa-300e-9665-1d55cdc1ff74,NaN,fb63318f-202c-43d3-b68b-9a3e16a91403,e77dc4a1-cf15-4807-a534-c8cf9ee8e7c0


In [209]:
print(len(df.dropna(subset=['deliveroo_id'])))

7688


In [138]:
def aggregate_rating(rating1,rating2,reviews_nr1,reviews_nr2):
    # A function to calculate average rating based on nr of reviews given
    # a nan or zero will not influence the total
    #if np.isnan(rating1):
    if isinstance(rating1, str):        
        rating1=0
        reviews_nr1=0
    if isinstance(rating2, str):
        rating2=0
        reviews_nr2=0
    reviews_nr1=int(reviews_nr1)
    reviews_nr2=int(reviews_nr2)        
    if pd.isnull(reviews_nr1) or isinstance(reviews_nr1, str):
        reviews_nr1=0      
    if pd.isnull(reviews_nr2) or isinstance(reviews_nr2, str):
        reviews_nr2=0
    total_reviews=reviews_nr1+reviews_nr2
    print(total_reviews)
    print('=====')
    if total_reviews==0:
        agg_rating=9999
    else:
        agg_rating=((rating1*reviews_nr1)+(rating2*reviews_nr2))/total_reviews
    return agg_rating, total_reviews

def aggregate_price(price1,price2,nr_datasets):
    # A function to calculate average price
    # inputs : values 1-4, with nans/anomalous values set to 0. nans should still not influence the result 
    #print(price1)
    #print(price2)
    #print(nr_datasets)
    if (pd.isnull(price2)) :
        agg_price_range=price1
        new_nr_datasets=nr_datasets
    elif price2 == 0:
        agg_price_range=price1
        new_nr_datasets=nr_datasets
    elif type(price2) == 'str':
        agg_price_range=price1
        new_nr_datasets=nr_datasets        
    else:
        new_nr_datasets=nr_datasets+1
        agg_price_range=(price1*nr_datasets+price2)/new_nr_datasets        
    return agg_price_range, new_nr_datasets
    
def transfer_price_level_eatnow(x):
    #renormalizes eatnow prices to the 1-4 schema
    #--> keneng : normalize properly to floats instead of integers        
    if x < 5:
        y=0
    elif x > 90:
        y=0
    elif 5 <= x < 20:
        y=1
    elif 20 <= x < 30:
        y=2
    elif 30 <= x < 40:
        y=3
    elif 40 <= x < 90:
        y=4
    return y

def replacethis(x):
    # replace schema for deliveroo and uber
    d ={"nan":"0","$":"1", "$$":"2", "$$$":"3","$$$$":"4"}
    encrypted_str = x.replace(x,d[x])
    return encrypted_str

In [197]:
print(len(df))
print(len(df.dropna()))

25320
156


In [198]:
listofall_dashids=list(df['dashmote_id'])
print(len(listofall_dashids))

25320


In [199]:
df_clean=df.copy()

# 1. Aggregate Price  Categories:

In [200]:
storesInfoDeliverooAU1 = pd.read_csv('DATA/storesInfoDeliverooAU1.csv')
storesInfoUbereatsAUNZ = pd.read_csv('DATA/final_storesInfoUbereatsAUNZ_2803cleaned.csv')
storesInfoEatnowAUNZ = pd.read_csv('DATA/final_storesInfoEatNowAUNZ_2803cleaned.csv')

storesgoogle = pd.read_csv('DATA/storeGoogleInfo2.csv',low_memory=False) 

In [201]:
# Reformat and scale the datasets to a 1-4 rating system:
storesInfoDeliverooAU1 = storesInfoDeliverooAU1.fillna('nan')
storesInfoUbereatsAUNZ = storesInfoUbereatsAUNZ.fillna('nan')
storesInfoDeliverooAU1['price_category_new']=storesInfoDeliverooAU1['price_category'].apply(lambda x: int(replacethis(x)))
storesInfoUbereatsAUNZ['price_category_new']=storesInfoUbereatsAUNZ['price_level'].apply(lambda x: int(replacethis(x)))
storesInfoEatnowAUNZ['price_category_new']=storesInfoEatnowAUNZ['price_level'].fillna(0).apply(lambda x: transfer_price_level_eatnow(x))
#storesInfoEatnowAUNZ['price_category_new']=storesInfoEatnowAUNZ['price_level'].apply(lambda x: int(replacethis(x)))


In [202]:
#create table from merging dash id and the first external data set:
big_table=pd.merge(df,storesInfoUbereatsAUNZ[['id_source','price_category_new']],left_on='uber_id',right_on='id_source',how='left')
print(len(big_table))

# merging first external data type and second data set:
big_table=pd.merge(big_table,storesInfoDeliverooAU1[['id_source','price_category_new']],left_on='deliveroo_id',right_on='id_source',how='left')
big_table=big_table.fillna(0)
# Aggregate rating is now based on 1 dataset
big_table['nr_datasets_price']=1

#Calculate new rating:
big_table['agg_price'], big_table['nr_datasets_price']=zip(*big_table.apply(lambda x: aggregate_price(x['price_category_new_x'],x['price_category_new_y'],x['nr_datasets_price']),axis=1))                                              
#big_table['agg_price'], big_table['nr_datasets']=zip(*big_table.apply(lambda x: aggregate_price(x['agg_price'],x['price_category_new_y'],x['nr_datasets']),axis=1))                                              
# Aggregate rating is now based on 2 datasets

# merging another data set:
big_table=pd.merge(big_table,storesInfoEatnowAUNZ[['id_source','price_category_new']],left_on='eatnow_id',right_on='id_source',how='left')
big_table=big_table.fillna(0)
# After all datasets are merged, Aggregate rating is now based on 3 datasets
# Calculate new rating:
big_table['agg_price'], big_table['nr_datasets_price']=zip(*big_table.apply(lambda x: aggregate_price(x['agg_price'],x['price_category_new_y'],x['nr_datasets_price']),axis=1))                                              

print(len(big_table))
big_table.head()

25320
25320


,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id,id_source_x,price_category_new_x,id_source_y,price_category_new_y,nr_datasets_price,agg_price,id_source,price_category_new
0,89977249-e023-405a-b533-3c1f33f804a2,5f04302c-68b1-3a83-8df9-0a05262b7200,0,40408a01-d9b3-443b-8e9e-53f8afb513f3,c26dafda-0437-4a47-9f79-6fb45541f1ea,0,0.0,5f04302c-68b1-3a83-8df9-0a05262b7200,3.0,3,2.0,0,0.0
1,9e7b397d-d615-4fb1-95d7-9d04d07765fd,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,0,0,1eb55afc-2faf-4c81-b6ff-5cdd4aa0f107,0,0.0,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,0.0,1,0.0,0,0.0
2,1e7ca034-a7c2-4a7c-8b3e-65bb4bf54da5,a80efa26-9e1d-31e3-88a5-b68d983be5c5,0,0,6f6f8fc6-1a3d-4e21-bfdc-45c1aa8b7577,0,0.0,a80efa26-9e1d-31e3-88a5-b68d983be5c5,0.0,1,0.0,0,0.0
3,9e829615-d0e6-446e-89cc-8940cb76f452,ca013261-a68e-3cbe-86fc-57c59f09940b,0,dbe938c5-eea7-4b0a-811d-145c679ba64a,ae702a3c-c50e-4c80-8db1-15e12a6837ad,0,0.0,ca013261-a68e-3cbe-86fc-57c59f09940b,0.0,1,0.0,0,0.0
4,1fc69820-eca5-4a86-b50b-db666ed2aef8,2326296b-c3fa-300e-9665-1d55cdc1ff74,0,fb63318f-202c-43d3-b68b-9a3e16a91403,e77dc4a1-cf15-4807-a534-c8cf9ee8e7c0,0,0.0,2326296b-c3fa-300e-9665-1d55cdc1ff74,0.0,1,0.0,0,0.0


In [203]:
df_clean=df.copy()
df_clean=pd.merge(df_clean,big_table[['dashmote_id','nr_datasets_price','agg_price']],on='dashmote_id',how='left')
df_clean.sort_values('nr_datasets_price',ascending=False)

,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id,nr_datasets_price,agg_price
0,89977249-e023-405a-b533-3c1f33f804a2,5f04302c-68b1-3a83-8df9-0a05262b7200,NaN,40408a01-d9b3-443b-8e9e-53f8afb513f3,c26dafda-0437-4a47-9f79-6fb45541f1ea,3,2.000000
3202,b7ea69d0-635a-4ea6-af38-143a79b5396c,69c02678-2627-386d-affd-3d46868bdeb1,NaN,7dbfed61-fb50-4fbc-8b7e-10bf561729e0,NaN,3,0.666667
3209,af06069a-161c-4aa4-b745-deb12dd1299b,ad127cbb-f079-3402-97e5-483fc4ef41bd,NaN,1f4b445b-a20e-4644-b64b-6ebebf540871,NaN,3,1.333333
3208,f0d058d2-2651-4b98-8cf2-77b99df47b21,2e62ef81-2d62-3d6e-a649-fc2988575e20,NaN,6adf38f1-dc42-4807-8aa6-07a84b02c15f,NaN,3,0.666667
3207,558e331c-a735-4553-91d8-20dac4c904dd,64a9251c-19d9-3637-8d52-4b5e6872a35c,NaN,6adf38f1-dc42-4807-8aa6-07a84b02c15f,NaN,3,1.333333
3206,566950f0-5a62-4a1d-8d6a-6c8510c65662,798ae357-ccb0-3065-aec8-c0eb22a90b88,NaN,6adf38f1-dc42-4807-8aa6-07a84b02c15f,NaN,3,0.666667
7595,8282c92b-5720-4c34-aef7-154463c5e657,f7e1db11-d773-3e4a-9acc-c0774363432b,NaN,NaN,NaN,3,2.000000
7597,206a0817-4204-4653-834c-23daf355f31a,386340b4-a645-3a7a-91b7-3dcb10a8ba48,NaN,NaN,NaN,3,0.666667
7598,d12a6824-674a-4fc3-8b4e-be7a7fdbec9d,e38d4f0c-32d7-3d24-8fa5-e0e8f0f9f05e,NaN,NaN,NaN,3,0.666667
3191,ffefcafd-a7de-4100-933b-87add853a696,4c2fef86-fbc4-3788-a302-84badcf8eb07,NaN,57b441c7-0850-43a8-b1a3-9dbbbf035aa8,NaN,3,1.333333


In [204]:
print(len(df_clean))

25320


In [205]:
print(len(df_clean.dropna(subset=['deliveroo_id'])))

7688


# 2. Aggregate Reviews, Ratings and Popularity :

In [148]:
#create table from merging dash id and the first external data set:
big_table=pd.merge(df,storesInfoUbereatsAUNZ[['id_source','price_category_new']],left_on='uber_id',right_on='id_source',how='left')
print(len(big_table)) 

25320


In [149]:
# #storesInfoDeliverooAU1 = pd.read_csv('DATA/storesInfoDeliverooAU1.csv')
# #storesInfoDeliverooAU = pd.read_csv('DATA/storesInfoDeliverooAU.csv')
# storesInfoUbereatsAUNZ = pd.read_csv('DATA/storesInfoUbereatsAUNZ.csv')
# storesInfoEatnowAUNZ = pd.read_csv('DATA/storesInfoEatNowAUNZ.csv')
# storesInfoDeliverooAU = pd.read_csv('../storesInfoDeliverooAU.csv')

In [150]:
storesInfoDeliverooAU1 = pd.read_csv('DATA/storesInfoDeliverooAU1.csv')
# storesInfoDeliverooAU = pd.read_csv('DATA/storesInfoDeliverooAU.csv')
# storesInfoUbereatsAUNZ = pd.read_csv('DATA/storesInfoUbereatsAUNZ.csv')
# storesInfoEatnowAUNZ = pd.read_csv('DATA/storesInfoEatNowAUNZ.csv')

#storesInfoDeliverooAU = pd.read_csv('DATA/final_storesInfoDeliverooAU_2803cleaned.csv')
storesInfoUbereatsAUNZ = pd.read_csv('DATA/final_storesInfoUbereatsAUNZ_2803cleaned.csv')
storesInfoEatnowAUNZ = pd.read_csv('DATA/final_storesInfoEatNowAUNZ_2803cleaned.csv')

storesgoogle = pd.read_csv('DATA/storeGoogleInfo2.csv',low_memory=False)

In [151]:
#mealsInfoUbereatsAUNZ = pd.read_csv('DATA/final_storesInfoEatNowAUNZ.csv')
storesInfoUbereatsAUNZ['reviews_nr']=storesInfoUbereatsAUNZ['reviews_nr'].fillna(0)
storesInfoUbereatsAUNZ['reviews_nr']=storesInfoUbereatsAUNZ['reviews_nr'].apply(lambda x: int(str(x).replace("+","")) )

So far used: Uber, Deliveroo, eatnow

In [152]:
#create table from merging dash id and the first external data set:
big_table=pd.merge(df,storesInfoUbereatsAUNZ[['id_source','rating','reviews_nr']],left_on='uber_id',right_on='id_source',how='left').fillna(0)
print(len(big_table))
# merging first external data type and second data set:
big_table=pd.merge(big_table,storesInfoDeliverooAU[['id_source','rating','reviews_nr']],left_on='deliveroo_id',right_on='id_source',how='left').fillna(0)
# Aggregate rating is now based on 1 dataset
big_table['total_reviews']=0
#Calculate new rating:
#big_table['agg_rating'], big_table['total_reviews']=zip(*big_table.apply(lambda x: aggregate_price(x['price_category_new_x'],x['price_category_new_y'],x['nr_datasets']),axis=1))                                              
big_table['agg_rating'], big_table['total_reviews']=zip(*big_table.apply(lambda x: aggregate_rating(x['rating_x'],x['rating_y'],x['reviews_nr_x'],x['reviews_nr_y']),axis=1)   )
print(big_table['total_reviews'].max())
# Aggregate rating is now based on 2 datasets
# merging another data set:
big_table=pd.merge(big_table,storesInfoEatnowAUNZ[['id_source','rating','reviews_nr']],left_on='eatnow_id',right_on='id_source',how='left').fillna(0)

#Calculate new rating:
#big_table['agg_rating'], big_table['total_reviews']=zip(*big_table.apply(lambda x: aggregate_price(x['price_category_new_x'],x['price_category_new_y'],x['nr_datasets']),axis=1))                                              
big_table['agg_rating'], big_table['total_reviews']=zip(*big_table.apply(lambda x: aggregate_rating(x['rating_x'],x['rating_y'],x['reviews_nr_x'],x['reviews_nr_y']),axis=1)   )
# Aggregate rating is now based on 3 datasets
print(big_table['total_reviews'].max())
print(len(big_table))
big_table.head()



25320
0
39
39
=====
0
0
0
=====
0
0
0
=====
0
30
30
=====
0
0
0
=====
0
50
50
=====
0
31
31
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
14
14
=====
0
27
27
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
21
21
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
28
28
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
32
32
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
19
19
=====
0
50
50
=====
0
12
12
=====
0
50
50
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
10
10
=====
0
24
24
=====
0
50
50
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
45
45
=====
0
10
10
=====
0
41
41
==

=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
60
50
110
=====
0
50
50
=====
0
36
36
=====
0
50
50
=====
0
50
50
=====
0
29
29
=====
0
50
50
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
21
21
=====
0
24
24
=====
0
50
50
=====
0
0
0
=====
0
13
13
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
20
20
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
==

0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
15
15
=====
0
0
0
=====
0
29
29
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
23
23
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
16
16
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
47
47
=====
0
0
0
=====
0
0
0
=====
0
22
22
=====
0
39
39
=====
0
14
14
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
11
11
=====
0
50
50
=====
0
50
50
=====
0
28
28
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
0
0
=====
0
22
22
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
11
11
=====
0
0
0
=====
0
50
50
=====
0
47
47
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====


=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
14
14
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
14
14
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
48
48
=====
0
50
50
=====
0
38
38
=====
0
40
40
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
0
0
=====
0
12
12
=====
0
30
30
=====
0
20
20
=====
0
0
0
=====
0
14
14
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0


0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
0
0
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
50
50
=====
0
35
35
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0


...

0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
==

0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0


0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0


0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
=====
0
0
0
==

,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id,id_source_x,rating_x,reviews_nr_x,id_source_y,rating_y,reviews_nr_y,total_reviews,agg_rating,id_source,rating,reviews_nr
0,89977249-e023-405a-b533-3c1f33f804a2,5f04302c-68b1-3a83-8df9-0a05262b7200,0,40408a01-d9b3-443b-8e9e-53f8afb513f3,c26dafda-0437-4a47-9f79-6fb45541f1ea,0,0.0,0.0,5f04302c-68b1-3a83-8df9-0a05262b7200,3.9,39.0,39,3.9,0,0.0,0.0
1,9e7b397d-d615-4fb1-95d7-9d04d07765fd,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,0,0,1eb55afc-2faf-4c81-b6ff-5cdd4aa0f107,0,0.0,0.0,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,nan,0.0,0,9999.0,0,0.0,0.0
2,1e7ca034-a7c2-4a7c-8b3e-65bb4bf54da5,a80efa26-9e1d-31e3-88a5-b68d983be5c5,0,0,6f6f8fc6-1a3d-4e21-bfdc-45c1aa8b7577,0,0.0,0.0,a80efa26-9e1d-31e3-88a5-b68d983be5c5,nan,0.0,0,9999.0,0,0.0,0.0
3,9e829615-d0e6-446e-89cc-8940cb76f452,ca013261-a68e-3cbe-86fc-57c59f09940b,0,dbe938c5-eea7-4b0a-811d-145c679ba64a,ae702a3c-c50e-4c80-8db1-15e12a6837ad,0,0.0,0.0,ca013261-a68e-3cbe-86fc-57c59f09940b,3.8,30.0,30,3.8,0,0.0,0.0
4,1fc69820-eca5-4a86-b50b-db666ed2aef8,2326296b-c3fa-300e-9665-1d55cdc1ff74,0,fb63318f-202c-43d3-b68b-9a3e16a91403,e77dc4a1-cf15-4807-a534-c8cf9ee8e7c0,0,0.0,0.0,2326296b-c3fa-300e-9665-1d55cdc1ff74,nan,0.0,0,9999.0,0,0.0,0.0


In [153]:
big_table['agg_rating'].loc[big_table['agg_rating']==0]=np.nan
big_table['agg_rating'].loc[big_table['agg_rating']==9999]=np.nan
big_table['agg_rating'].loc[big_table['total_reviews']==0]=np.nan

/jet/var/python/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [154]:
big_table.head()

,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id,id_source_x,rating_x,reviews_nr_x,id_source_y,rating_y,reviews_nr_y,total_reviews,agg_rating,id_source,rating,reviews_nr
0,89977249-e023-405a-b533-3c1f33f804a2,5f04302c-68b1-3a83-8df9-0a05262b7200,0,40408a01-d9b3-443b-8e9e-53f8afb513f3,c26dafda-0437-4a47-9f79-6fb45541f1ea,0,0.0,0.0,5f04302c-68b1-3a83-8df9-0a05262b7200,3.9,39.0,39,3.9,0,0.0,0.0
1,9e7b397d-d615-4fb1-95d7-9d04d07765fd,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,0,0,1eb55afc-2faf-4c81-b6ff-5cdd4aa0f107,0,0.0,0.0,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,nan,0.0,0,NaN,0,0.0,0.0
2,1e7ca034-a7c2-4a7c-8b3e-65bb4bf54da5,a80efa26-9e1d-31e3-88a5-b68d983be5c5,0,0,6f6f8fc6-1a3d-4e21-bfdc-45c1aa8b7577,0,0.0,0.0,a80efa26-9e1d-31e3-88a5-b68d983be5c5,nan,0.0,0,NaN,0,0.0,0.0
3,9e829615-d0e6-446e-89cc-8940cb76f452,ca013261-a68e-3cbe-86fc-57c59f09940b,0,dbe938c5-eea7-4b0a-811d-145c679ba64a,ae702a3c-c50e-4c80-8db1-15e12a6837ad,0,0.0,0.0,ca013261-a68e-3cbe-86fc-57c59f09940b,3.8,30.0,30,3.8,0,0.0,0.0
4,1fc69820-eca5-4a86-b50b-db666ed2aef8,2326296b-c3fa-300e-9665-1d55cdc1ff74,0,fb63318f-202c-43d3-b68b-9a3e16a91403,e77dc4a1-cf15-4807-a534-c8cf9ee8e7c0,0,0.0,0.0,2326296b-c3fa-300e-9665-1d55cdc1ff74,nan,0.0,0,NaN,0,0.0,0.0


In [155]:
len(big_table)

25320

In [156]:
df_clean=pd.merge(df_clean,big_table[['dashmote_id','agg_rating','total_reviews']],on='dashmote_id',how='left')
df_clean.sort_values('total_reviews',ascending=False)

,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id,nr_datasets_price,agg_price,agg_rating,total_reviews
709,a32bb7ce-f6a3-4634-8e38-2a3f3a772c84,83e10fc1-b115-3e75-adab-40fd92a41443,NaN,d2640c2d-a98c-45df-b8c4-a34afc53265b,2bdcaa4f-23c4-4549-8b97-03763750de0b,3,1.666667,4.498258,550
6824,bc53274b-680e-4986-b779-670f12524ff6,9a53fbb7-c85e-3c9a-9045-7376300c9670,99dd8a58-692e-3379-97f5-84c859eee6b7,40662dab-d4f0-4ce0-8699-9e3168e9e941,50c089cf-cb47-4d33-8ef9-4ef3668c6ef9,3,1.333333,4.556061,550
492,4a90ca7f-4a3d-427b-9df7-98dd026f112d,d12f48b0-dbb3-36b3-9882-5d7386ab0cd0,NaN,d39ea386-e882-47b2-951f-434db808ccb7,1c99a280-cccd-40b3-b7ff-2bd37f5d1df2,1,1.000000,4.145001,550
451,e9b2df61-fbd6-42c7-a18f-d168dc8253eb,935466b5-6f11-3219-8567-eba3d96cc240,NaN,NaN,f87df704-c846-4127-b733-3835db6d4726,1,1.000000,4.624108,550
469,1203e6bd-9571-450d-bef7-87fb9f5ab3a3,4feed063-0d09-378f-8811-7e92f40591a4,NaN,b990e368-3a90-498d-a1b9-3265ca8fb7c5,5424813f-4ba4-448e-88c7-75d0f2a73fae,1,1.000000,4.536075,550
296,3a79a794-832c-4240-b459-bb3291d542f7,a5302289-7e4a-395b-a35f-8a1455ab57d6,NaN,5d1e58c2-92d3-4da9-97d3-904b068eb381,aae330ba-a664-4d51-ab0a-a014a30cbbc6,1,1.000000,4.596419,550
401,e58bf2f4-87eb-4b15-ac0f-50d36d5f00ab,5b458364-6c26-3b03-9d71-c01a6ce7f077,NaN,a21b961c-bfa4-4769-beab-4014ae72927c,d33496f9-9426-4bca-9b5b-1f164a357186,1,1.000000,4.307656,550
441,57805417-542c-4fb8-a01c-24fb1dc35d2a,f0b985f8-34fb-378c-936f-7c17fdb3fac6,NaN,9357af84-a948-4904-b8c5-ebbc2c8d9abe,285d29e6-ba61-4491-bc51-7020446a4e0d,1,1.000000,4.442700,550
456,6f44aeac-533e-4a0d-834f-2a51d106255d,c729a506-6a2e-3150-b8c1-ad6916bb8d3e,NaN,729a69b9-8c52-4c53-91d0-c5b7c21fd672,779313c2-ab8d-4881-8aea-3872c6309a70,1,1.000000,4.458678,550
6736,d6e971cc-4c2b-49a8-8109-ce0589494fa3,9eb0b19d-c268-3faa-a11d-6f128afa2cf3,5354f649-b553-383f-9e24-57e7aafb5b91,bb3c2f0c-c436-4b89-807d-909b35f0aa5b,eaa583d1-257d-4701-a274-c8fd74760ae5,3,1.666667,4.047419,550


# 3. Aggregate kitchen info :

This is temporary only: this should be done after aggreation. Today we only use the deliveroo data

In [157]:
# storesInfoDeliverooAU = pd.read_csv('DATA/final_storesInfoDeliverooAU_2803cleaned.csv')
# storesInfoUbereatsAUNZ = pd.read_csv('DATA/final_storesInfoUbereatsAUNZ_2803cleaned.csv')
# storesInfoEatnowAUNZ = pd.read_csv('DATA/final_storesInfoEatNowAUNZ_2803cleaned.csv')

In [158]:
kitchen_deliv=pd.read_csv('DATA/final_storesInfoDeliverooAU_2803cleaned.csv')
kitchen_deliv.columns

Index(['Unnamed: 0', 'address', 'city', 'country', 'house_number', 'id_source',
       'kitchen', 'lat', 'lon', 'name', 'phone', 'postal_code', 'rating',
       'reviews_nr', 'source', 'source_address', 'street', 'url',
       'deliveroo_country', 'deliveroo_food'],
      dtype='object')

In [159]:
kitchen_deliv['deliveroo_food'].unique()

array(['Burgers', nan, 'Noodles', 'Pizza', 'Dessert', 'Curry', 'Salads',
       'Vegan', 'Seafood', 'Sushi'], dtype=object)

In [160]:
kitchen_deliv1=kitchen_deliv[['id_source', 'deliveroo_country', 'deliveroo_food']]

In [161]:
kitchen_deliv1[ 'deliveroo_country'].unique()

array(['American', 'Japanese', 'Thai', 'Chinese', 'Greek', 'Vietnamese',
       'Italian', 'Indian', 'Turkish', 'Malaysian', 'Korean', 'Mexican',
       nan, 'Australian', 'Lebanese', 'African', 'Nepalese', 'French',
       'Indonesian', 'Sri Lankan'], dtype=object)

In [162]:
kitchen_deliv1[ 'deliveroo_food'].unique()

array(['Burgers', nan, 'Noodles', 'Pizza', 'Dessert', 'Curry', 'Salads',
       'Vegan', 'Seafood', 'Sushi'], dtype=object)

In [163]:
len(df_clean)

25320

In [164]:
df_clean0=pd.merge(df_clean,kitchen_deliv1,left_on='deliveroo_id',right_on='id_source',how='left')
df_clean0=df_clean0.rename(columns={'deliveroo_country' : 'kitchen_country' , 'deliveroo_food':'kitchen_type'})
df_clean0.head()

,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id,nr_datasets_price,agg_price,agg_rating,total_reviews,id_source,kitchen_country,kitchen_type
0,89977249-e023-405a-b533-3c1f33f804a2,5f04302c-68b1-3a83-8df9-0a05262b7200,NaN,40408a01-d9b3-443b-8e9e-53f8afb513f3,c26dafda-0437-4a47-9f79-6fb45541f1ea,3,2.0,3.9,39,5f04302c-68b1-3a83-8df9-0a05262b7200,Indian,Vegan
1,9e7b397d-d615-4fb1-95d7-9d04d07765fd,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,NaN,NaN,1eb55afc-2faf-4c81-b6ff-5cdd4aa0f107,1,0.0,NaN,0,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,NaN,NaN
2,1e7ca034-a7c2-4a7c-8b3e-65bb4bf54da5,a80efa26-9e1d-31e3-88a5-b68d983be5c5,NaN,NaN,6f6f8fc6-1a3d-4e21-bfdc-45c1aa8b7577,1,0.0,NaN,0,a80efa26-9e1d-31e3-88a5-b68d983be5c5,NaN,Dessert
3,9e829615-d0e6-446e-89cc-8940cb76f452,ca013261-a68e-3cbe-86fc-57c59f09940b,NaN,dbe938c5-eea7-4b0a-811d-145c679ba64a,ae702a3c-c50e-4c80-8db1-15e12a6837ad,1,0.0,3.8,30,ca013261-a68e-3cbe-86fc-57c59f09940b,Italian,Pizza
4,1fc69820-eca5-4a86-b50b-db666ed2aef8,2326296b-c3fa-300e-9665-1d55cdc1ff74,NaN,fb63318f-202c-43d3-b68b-9a3e16a91403,e77dc4a1-cf15-4807-a534-c8cf9ee8e7c0,1,0.0,NaN,0,2326296b-c3fa-300e-9665-1d55cdc1ff74,Nepalese,Curry


In [165]:
df_clean0['kitchen_type'].unique()

array(['Vegan', nan, 'Dessert', 'Pizza', 'Curry', 'Sushi', 'Burgers',
       'Noodles', 'Seafood', 'Salads'], dtype=object)

In [166]:
len(df_clean0)

25320

# 4. Add Menu info :

In [167]:
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

#storesInfoDeliverooAU = pd.read_csv('data_csv/storesInfoDeliverooAU.csv')
#storesInfoEatNowAUNZ = pd.read_csv('data_csv/storesInfoEatNowAUNZ.csv')
#storesInfoUbereatsAUNZ=pd.read_csv('data_csv/storesInfoUbereatsAUNZ.csv')

#mIDAU=pd.read_csv('DATA/mIDAU.csv')
#mIEAUNZ=pd.read_csv('DATA/mIEAUNZ.csv')
mIUAUNZ=pd.read_csv('DATA/mIUAUNZ.csv')

mIDAU=pd.read_csv('DATA/d_2083.csv',low_memory=False)[['id_source','Brand','Coca-Cola','Subbrands','is_coca']]
mIEAUNZ=pd.read_csv('DATA/e_2083.csv',low_memory=False)[['id_source','Brand','Coca-Cola','Subbrands','is_coca']]
mIUAUNZ=pd.read_csv('DATA/u_2083.csv',low_memory=False)[['id_source','Brand','Coca-Cola','Subbrands','is_coca']]


In [169]:
mIEAUNZ.head()


,id_source,Brand,Coca-Cola,Subbrands,is_coca
0,b58b8619-76d8-3480-bb67-4c5daa30c8eb,Powerade,Coca-Cola,['Powerade'],1
1,a2ff4952-642b-37f6-bb99-80af1da8f260,Sprite,Coca-Cola,['Sprite'],1
2,1e9ca3ed-06b2-3542-98e7-fb87b1c0844a,Fanta,Coca-Cola,"['Fanta', 'Fanta Orange', 'Fanta Cassis', 'Fan...",1
3,2ed64243-974d-3615-a3a6-6149cd80928d,Sprite,Coca-Cola,['Sprite'],1
4,134e922a-c75c-3837-b079-56500af255aa,Sprite,Coca-Cola,['Sprite'],1


In [170]:
len(mIDAU)

563165

In [171]:
mIDAU.groupby('Coca-Cola').count()['id_source']

Coca-Cola
Coca-Cola    3014
Other        1218
Name: id_source, dtype: int64

In [172]:
mIDAU.groupby('is_coca').count()['id_source']

is_coca
0    560151
1      3014
Name: id_source, dtype: int64

In [173]:
len(list(mIDAU['Brand'].unique()))

32

In [174]:
mIDAU.groupby('Brand').count()['id_source']

Brand
7 Up               182
Acqua Panna          2
Appletiser           9
Arizona              1
Budweiser            5
Carlsberg            2
Coca-Cola           43
Coors                4
Dr Pepper           16
Evian                2
Fanta             1187
Fuze Tea            16
Gatorade            41
Guinness             1
Heineken            42
Irn Bru              3
Lipton              10
Lucozade             3
Monster             19
Naked               24
Orangina             2
Pepsi              761
Perrier              3
Powerade           121
RedBull              1
Ribena               7
San Pellegrino      47
Schweppes           21
Sprite            1617
Stella Artois       23
Tropicana           17
Name: id_source, dtype: int64

In [175]:
mIDAU.describe()

,is_coca
count,563165.000000
mean,0.005352
std,0.072961
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [176]:
#mIDAU=pd.read_csv('DATA/mIDAU.csv')
mIDAU[mIDAU['Brand']=='Fanta'][:4]


,id_source,Brand,Coca-Cola,Subbrands,is_coca
0,2d07caaf-19a1-3eb5-93c7-849cddf0b43e,Fanta,Coca-Cola,"['Fanta', 'Fanta Orange', 'Fanta Cassis', 'Fan...",1
2,9273bb01-0882-3b70-802f-cf3f722ec1d3,Fanta,Coca-Cola,"['Fanta', 'Fanta Orange', 'Fanta Cassis', 'Fan...",1
3,d74407b9-a3d2-3f7f-ba21-3611c45d93a6,Fanta,Coca-Cola,"['Fanta', 'Fanta Orange', 'Fanta Cassis', 'Fan...",1
5,45bb6dba-18d1-3a16-a846-6c8d2f7f947a,Fanta,Coca-Cola,"['Fanta', 'Fanta Orange', 'Fanta Cassis', 'Fan...",1


In [177]:
def store_test1(store_df,name_df):
    list_=name_df['id_source'].unique().tolist()
    store_df['coca_result']=store_df.progress_apply(
    lambda x:1 if x['id_source'] in(list_) else 0, axis=1)
    store_df = store_df.sort_values(by=["coca_result"],ascending=False)
    return store_df 

# def store_test(store_df,meal_df):
#     k=store_df["id_source"].isin(meal_df["id_source"].unique().tolist())
#     return k

In [178]:
df_clean.head()

,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id,nr_datasets_price,agg_price,agg_rating,total_reviews
0,89977249-e023-405a-b533-3c1f33f804a2,5f04302c-68b1-3a83-8df9-0a05262b7200,NaN,40408a01-d9b3-443b-8e9e-53f8afb513f3,c26dafda-0437-4a47-9f79-6fb45541f1ea,3,2.0,3.9,39
1,9e7b397d-d615-4fb1-95d7-9d04d07765fd,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,NaN,NaN,1eb55afc-2faf-4c81-b6ff-5cdd4aa0f107,1,0.0,NaN,0
2,1e7ca034-a7c2-4a7c-8b3e-65bb4bf54da5,a80efa26-9e1d-31e3-88a5-b68d983be5c5,NaN,NaN,6f6f8fc6-1a3d-4e21-bfdc-45c1aa8b7577,1,0.0,NaN,0
3,9e829615-d0e6-446e-89cc-8940cb76f452,ca013261-a68e-3cbe-86fc-57c59f09940b,NaN,dbe938c5-eea7-4b0a-811d-145c679ba64a,ae702a3c-c50e-4c80-8db1-15e12a6837ad,1,0.0,3.8,30
4,1fc69820-eca5-4a86-b50b-db666ed2aef8,2326296b-c3fa-300e-9665-1d55cdc1ff74,NaN,fb63318f-202c-43d3-b68b-9a3e16a91403,e77dc4a1-cf15-4807-a534-c8cf9ee8e7c0,1,0.0,NaN,0


In [179]:
sIDAU=store_test1(storesInfoDeliverooAU,mIDAU)[['id_source','coca_result']].rename(columns={'coca_result':'cola_deliveroo'})
sIUAUNZ=store_test1(storesInfoUbereatsAUNZ,mIUAUNZ)[['id_source','coca_result']].rename(columns={'coca_result':'cola_uber'})
sIEAUNZ=store_test1(storesInfoEatnowAUNZ,mIEAUNZ)[['id_source','coca_result']].rename(columns={'coca_result':'cola_eatnow'})



In [180]:
len(storesInfoDeliverooAU)

4945

In [181]:
sIDAU['cola_deliveroo'].sum()

4912

In [182]:
len(sIDAU['cola_deliveroo'])

4945

In [184]:
#sIDAU.groupby('Coca-Cola').count()['id_source']

In [185]:
df_clean1=pd.merge(df_clean0,sIDAU,left_on='deliveroo_id',right_on='id_source',how='left')
df_clean1=pd.merge(df_clean1,sIUAUNZ,left_on='uber_id',right_on='id_source',how='left')
df_clean1=pd.merge(df_clean1,sIEAUNZ,left_on='eatnow_id',right_on='id_source',how='left')

df_clean1['cola_present']=df_clean1[['cola_deliveroo','cola_uber','cola_eatnow']].fillna(0).sum(axis=1)
df_clean1=df_clean1.drop(['id_source_x','cola_deliveroo','id_source_y','cola_uber','id_source','cola_eatnow'],1)
df_clean1.head()

,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id,nr_datasets_price,agg_price,agg_rating,total_reviews,kitchen_country,kitchen_type,cola_present
0,89977249-e023-405a-b533-3c1f33f804a2,5f04302c-68b1-3a83-8df9-0a05262b7200,NaN,40408a01-d9b3-443b-8e9e-53f8afb513f3,c26dafda-0437-4a47-9f79-6fb45541f1ea,3,2.0,3.9,39,Indian,Vegan,1.0
1,9e7b397d-d615-4fb1-95d7-9d04d07765fd,99deb4f2-05d6-3c95-ae7d-5aed185df8a3,NaN,NaN,1eb55afc-2faf-4c81-b6ff-5cdd4aa0f107,1,0.0,NaN,0,NaN,NaN,1.0
2,1e7ca034-a7c2-4a7c-8b3e-65bb4bf54da5,a80efa26-9e1d-31e3-88a5-b68d983be5c5,NaN,NaN,6f6f8fc6-1a3d-4e21-bfdc-45c1aa8b7577,1,0.0,NaN,0,NaN,Dessert,1.0
3,9e829615-d0e6-446e-89cc-8940cb76f452,ca013261-a68e-3cbe-86fc-57c59f09940b,NaN,dbe938c5-eea7-4b0a-811d-145c679ba64a,ae702a3c-c50e-4c80-8db1-15e12a6837ad,1,0.0,3.8,30,Italian,Pizza,1.0
4,1fc69820-eca5-4a86-b50b-db666ed2aef8,2326296b-c3fa-300e-9665-1d55cdc1ff74,NaN,fb63318f-202c-43d3-b68b-9a3e16a91403,e77dc4a1-cf15-4807-a534-c8cf9ee8e7c0,1,0.0,NaN,0,Nepalese,Curry,1.0


In [217]:
df_clean1['cola_present'].sum()
print(100*(df_clean1['cola_present'].sum())/len(df_clean1))

25.102685624


In [216]:
temp=df_clean1.dropna(subset=['deliveroo_id']).drop_duplicates()

print(temp['cola_present'].sum())
print(100*(temp['cola_present'].sum())/len(temp))

5633.0
73.2700312175


In [215]:
temp=df_clean1.dropna(subset=['uber_id']).drop_duplicates()
print(len(temp))
print(temp['cola_present'].sum())
print(100*(temp['cola_present'].sum())/len(temp))

2594
2161.0
83.3076329992


In [214]:
temp=df_clean1.dropna(subset=['eatnow_id']).drop_duplicates()
print(len(temp))
print(temp['cola_present'].sum())
print(100*(temp['cola_present'].sum())/len(temp))

821
503.0
61.2667478685


In [220]:
temp=df_clean1.dropna(subset=['google_id']).drop_duplicates()
print(len(temp))
print(temp['cola_present'].sum())
print(100*(temp['cola_present'].sum())/len(temp))

22750
3886.0
17.0813186813


In [221]:
temp

,dashmote_id,deliveroo_id,eatnow_id,google_id,uber_id,nr_datasets_price,agg_price,agg_rating,total_reviews,kitchen_country,kitchen_type,cola_present
0,89977249-e023-405a-b533-3c1f33f804a2,5f04302c-68b1-3a83-8df9-0a05262b7200,NaN,40408a01-d9b3-443b-8e9e-53f8afb513f3,c26dafda-0437-4a47-9f79-6fb45541f1ea,3,2.000000,3.9,39,Indian,Vegan,1.0
3,9e829615-d0e6-446e-89cc-8940cb76f452,ca013261-a68e-3cbe-86fc-57c59f09940b,NaN,dbe938c5-eea7-4b0a-811d-145c679ba64a,ae702a3c-c50e-4c80-8db1-15e12a6837ad,1,0.000000,3.8,30,Italian,Pizza,1.0
4,1fc69820-eca5-4a86-b50b-db666ed2aef8,2326296b-c3fa-300e-9665-1d55cdc1ff74,NaN,fb63318f-202c-43d3-b68b-9a3e16a91403,e77dc4a1-cf15-4807-a534-c8cf9ee8e7c0,1,0.000000,NaN,0,Nepalese,Curry,1.0
5,6429b5f2-e678-41c0-9ff9-d29024216a6f,90c0674d-138f-3aed-9e61-b9580bf9c04e,NaN,55779856-693e-43a7-bf1e-3d5bc60c31c8,6dd434cb-364d-4b07-b855-3020dd9550ac,3,0.666667,4.6,50,Japanese,Sushi,1.0
6,230fb747-04cd-41af-a26d-3a725739b029,96134935-b31b-37e1-a8b4-a371ad3bbec6,NaN,0ab1f278-a404-426a-999e-97449b9fda33,329571da-4fb2-4e79-8d6a-f34baf780068,1,0.000000,3.9,31,Indonesian,Vegan,1.0
7,9be34d63-1445-46f8-91e6-8423c3966bbd,d0ed07d3-ec47-3809-8133-8e2630378cbf,NaN,a7ffd663-3d12-44be-bcc2-9b5ce94339d1,7ee46d45-fd36-442b-921b-b29d27a20c5e,3,0.666667,4.4,50,NaN,Burgers,1.0
8,16b1894e-2585-4270-8ca7-ab701a683772,1bb70bf9-56b4-3be9-a4a0-f2e58b1fd38e,NaN,272ae6e3-8b51-43dc-97b7-83f83c87f0be,d92a6812-9979-4391-b59d-68afaf39b169,3,1.333333,4.2,50,Turkish,Burgers,1.0
13,03203394-ec32-46d9-8aae-cea0ef9fec32,2b7dec6a-f156-30f3-96e4-436f86838c14,NaN,a4b4394c-f14f-490a-a313-c3632b689e37,6f10753e-c9c0-4b2b-bbe1-cd762d66be03,3,2.000000,NaN,0,Chinese,NaN,1.0
14,b98a6bbc-c7be-484f-b014-4bb97ac690f6,bef15894-0efd-3141-b9d8-f9421affe979,NaN,53b6c860-4670-4b7b-bf84-2efec44819b2,7ccb84c8-d191-462e-a3d1-f5a3dcf4528c,1,0.000000,4.4,50,Thai,Noodles,1.0
17,76ae709c-99f3-4a29-887d-acde634e7471,4b71c5e8-b772-3a5b-95e2-dff507f6c2b0,NaN,317ef8f9-89a4-4309-be08-77911676ea30,f09fe3c9-fcd9-42b7-adb0-3f1a56105c9e,3,2.000000,4.3,14,Italian,NaN,1.0


In [222]:
df_clean1['cola_present'].sum()/21604

0.29420477689316793

In [223]:
len(df_clean1) 

25320

In [224]:
df_clean1.columns

Index(['dashmote_id', 'deliveroo_id', 'eatnow_id', 'google_id', 'uber_id',
       'nr_datasets_price', 'agg_price', 'agg_rating', 'total_reviews',
       'kitchen_country', 'kitchen_type', 'cola_present'],
      dtype='object')